# American Option Pricing using Binomial Tree, with Automatic Differentiation for IV Estimation

This notebook tutorial is a walkthrough for pricing American options using the binomial model. It should be referred to alongside the mathematical proofs and tutorial side (Proofs), as it would be easier to understand. 

We use a classic binomial model to price American options, with the Cox-Ross-Rubinstein (CRR) method of parameter estimation (estimation of u and hence p using sigma). The binomial model is a simple, intuitive and effective way to price options, and it is a good starting point for understanding the mechanics of option pricing under risk-neutral, no-arbitrage assumptions. We do this through a vectorization approach, which is much faster than the standard approach of pricing each option separately and iteratively backwards, which will be explained in Proofs.

In addition to that, based on the paper "Using the Newton-Raphson Method with Automatic Differentiation to Numerically Solve the Implied Volatility of Stock Options via the Binomial Model" by Michael Klibanov et al. [https://arxiv.org/html/2207.09033v3], we use the Newton-Raphson method and the automatic differentiation technique to solve for the implied volatility of the option in a way that is computationally faster than Brent's root search method (secant and bisection method), with high accuracy (RMSE from Brent's root search method is almost 0).

### Workflow
1. Fetch Option Chain Data
2. Fetch Risk Free Rates
3. Calculate IV from Market Data
4. Plot Surface
5. Evaluation

### Basic Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from fredapi import Fred
import datetime as dt
from scipy.stats import norm
from scipy.interpolate import interp1d, griddata
from scipy.optimize import brentq
import time
import datetime
from datetime import timedelta, date
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error

### 1. Fetch Option Data

In [2]:
def option_chains(ticker, T_start, T_end, type, price_range = 0.30, threshold_volume = 10):
    """
    Download all option chains with expiration up to T, number of years ahead, 
    for a given ticker and condense into a dataframe and filters out options 
    with low volume or NAN volume.

    Workflow:
    1. Get market price of option now (S0)
    2. Get all option chains with expiration up to T
    3. Filter out options with low volume or NAN volume

    Parameters
    ----------
    ticker : str
        Ticker symbol of the stock
    T_start : float
        Number of years ahead to start filtering options
    T_end : float
        End of time range to filter options
    type : str
        Option type, 'c' for calls, 'p' for puts
    price_range : float, optional
        Range of strike prices to consider from current market price, by default 0.15
    threshold_volume : int, optional
        Minimum volume to include an option, by default 10

    Returns
    -------
    pd.DataFrame
        DataFrame containing the option chains with columns ['K', 'f', 'T', 'Volume', 'S0']
        for easy vector operations
    """
    yticker = yf.Ticker(ticker)
    expirations = yticker.options
    try:
        S0 = yticker.fast_info['last_price']
    except:
        # fallback if fast_info fails, slower
        S0 = yticker.history(period='1d')['Close'].iloc[-1]
    option_chain_list = []
    for dates in range(len(expirations)):
        time_to_expiry = (datetime.datetime.strptime(expirations[dates], '%Y-%m-%d') - datetime.datetime.today()).days / 365
        if time_to_expiry < T_start: # skip options that have T < T_start
            continue
        if time_to_expiry > T_end: # break when T > T_end
            break
        if type == 'c':
            option_week_df = yticker.option_chain(str(expirations[dates])).calls
        elif type == 'p':
            option_week_df = yticker.option_chain(str(expirations[dates])).puts
        else:
            raise ValueError('Invalid option type, only accepts c or p')
        if option_week_df.empty:
            print(f"No options found for {ticker} at {expirations[dates]}, skipping...")
            continue
        option_week_df['T'] = time_to_expiry
        option_chain_list.append(option_week_df[['strike', 'lastPrice', 'T', 'volume']])
        print(f"Obtained options for {ticker} at {expirations[dates]}")

        time.sleep(1) # prevent yfinance from blocking me
    option_chain = pd.concat(option_chain_list, ignore_index=True)

    # look at options centered around range about the current price
    option_chain_ranged = option_chain[(option_chain['strike'] / S0 - 1).abs() <= price_range]

    option_chain_ranged['S0'] = S0
    option_chain_ranged.columns = ['K', 'f', 'T', 'Volume','S0']

    # remove illiquid options, filter out options with below threshold volume
    # open interest data from yahoo is not reliable, hence we ignore it and use volume only
    option_chain_ranged.dropna(subset=['Volume'], inplace=True) # dropna for np.percentile to work
    option_chain_ranged = option_chain_ranged[(option_chain_ranged['Volume'] >= threshold_volume) & (option_chain_ranged['T'] > 0)]

    option_chain_ranged.reset_index(drop = True, inplace = True)

    return option_chain_ranged

In [3]:
option_chains = option_chains('AAPL', 0.05, 1, price_range = 0.4, type = 'c')
option_chains

Obtained options for AAPL at 2026-01-30
Obtained options for AAPL at 2026-02-06
Obtained options for AAPL at 2026-02-13
Obtained options for AAPL at 2026-02-20
Obtained options for AAPL at 2026-03-20
Obtained options for AAPL at 2026-04-17
Obtained options for AAPL at 2026-05-15
Obtained options for AAPL at 2026-06-18
Obtained options for AAPL at 2026-07-17
Obtained options for AAPL at 2026-08-21
Obtained options for AAPL at 2026-09-18
Obtained options for AAPL at 2026-12-18


C:\Users\samuel\AppData\Local\Temp\ipykernel_40056\3255860128.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  option_chain_ranged['S0'] = S0
C:\Users\samuel\AppData\Local\Temp\ipykernel_40056\3255860128.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  option_chain_ranged.dropna(subset=['Volume'], inplace=True) # dropna for np.percentile to work


,K,f,T,Volume,S0
0,225.0,42.39,0.057534,47.0,260.329987
1,230.0,32.16,0.057534,14.0,260.329987
2,235.0,26.54,0.057534,120.0,260.329987
3,240.0,22.18,0.057534,10.0,260.329987
4,245.0,18.00,0.057534,79.0,260.329987
...,...,...,...,...,...
248,320.0,9.20,0.939726,12.0,260.329987
249,330.0,7.24,0.939726,19.0,260.329987
250,340.0,5.75,0.939726,27.0,260.329987
251,350.0,4.56,0.939726,61.0,260.329987


### 2. Fetch Risk Free Rates

We use Fred API to extract latest money market rates from Fred as Risk Free Rate. This is because money market rates are considered to be short rates in the US, with investment horizon typically of 3 months to 1 year. Therefore, as options typicallyare short term contracts, we use money market rates as risk free rates.

In [4]:
with open(r"FRED_api_key.txt", 'r') as file:
    api_key = file.read()
fred = Fred(api_key = api_key)
# lookback window cannnot be too wide because FRED data is not updated daily
start_fred = (datetime.datetime.today() - datetime.timedelta(days = 30)).strftime('%Y-%m-%d')
end_fred = datetime.datetime.today().strftime('%Y-%m-%d')
# DCPF1M is the money market rate prevailiing in the US.
money_market_rates = fred.get_series("DCPF1M", observation_start = start_fred, observation_end = end_fred)
money_market_rates.dropna(inplace = True)

rf = money_market_rates.iloc[-1] / 100 # find the latest value of the series
print("Current Money Market Rates as of ", money_market_rates.index[-1].strftime('%Y-%m-%d'), "is: ", rf)

Current Money Market Rates as of  2026-01-06 is:  0.0364


### 3. Calculate IV from Market Data

We first define the functions binomial_price and binomial_price_vega_ad. These will be our "engines" for a recursive method of calculation of the price (and vega, for the latter function) of the option; that means to calculate the option price today by backwards calculating the future outcomes of option values and discounting to today. Please do refer to Proofs for the full explanation on the mechanics.

#### Core functions

In [5]:
def binomial_price(S0, K, T, r, sigma, type='c'):
    """
    We calculate the price of option using binomial approach. 
    We assume that the stock price can move up or down by a factor of u or d in each time step, where
    d = 1/u, the reciprocal. This is to ensure that at N time steps, there will be N+1 possible outcomes.

    By default, number of time steps N = 5000, to try to assume continuous time when dt = T/N is small.
    For time complexity, setting N at a higher value will raise the calculation time significantly.

    For parameters u and d estimation, we use the Cox, Ross and Rubinstein (CRR) method, where:
    u = e^(sigma * sqrt(dt))
    d = 1/u
    p = (e^(r * dt) - d) / (u - d)

    We use a vectorized approach to a recursive method to calculate the option price.

    Args:
        S0 (float): Current stock price.
        K (float): Option strike price.
        T (float): Time to expiration in years.
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock.
        type (str): Option type, 'c' for call, 'p' for put.

    Returns:
    float: Option price.
    """
    N = round(5000 * T)
    dt = T / N # time step per layer should tend to 0 to assume continuous time

    a = np.exp(- r * dt) # discount by risk free rate
    
    u = np.exp(sigma * np.sqrt(dt)) # up state
    d = 1 / u # down state is reciprocal of up state
    p = (np.exp(r * dt) - d) / (u - d) # probability of up state

    # Array of stock prices at maturity, where t = T
    # s_ij = S_0 * u^(i) * d^(j) where i + j = N, hence:
    S_t = S0 * u ** np.arange(N,-1,-1) * d ** np.arange(0,N+1,1) # (N+1) x 1 vector

    # Hence, option prices at time T, the Nth step is as follows:
    if type == 'c':
        f = np.maximum(S_t - K, 0)
    elif type == 'p':
        f = np.maximum(K - S_t, 0)
    else:
        raise ValueError("Option type must be 'c' or 'p'.")
    
    # recursive calculation of option price at each time step, from N-1 to 0
    for i in range(N-1, -1, -1):
        S_t = S0 * u ** np.arange(i,-1,-1) * d ** np.arange(0,i+1,1) # N number of outcomes at N-1 step
        # value of options at previous step = discount * (p * all up steps * (1-p) * all down steps
        f_previous = a * (p * f[:-1] + (1-p) * f[1:]) # N number of outcomes at N-1 step
        f = f_previous

        # as american option has the ability to exercise at any time, it is always worth
        # at least the intrinsic value of the option, S_t - k for call, k - S_t for put
        if type == 'c':
            f = np.maximum(f, S_t - K)
        elif type == 'p':
            f = np.maximum(f, K - S_t)

    return f[0]


In [6]:
def binomial_price_vega_ad(S0, K, T, r, sigma, type='c'):
    """
    This is an extension to the binomial_price function through simultaneous calculation of vega.
    At the same time of option price calculation, we also calculate the vega of the option through automatic
    differentiation method.

    Args:
        S0 (float): Current stock price.
        K (float): Option strike price.
        T (float): Time to expiration in years.
        r (float): Risk-free interest rate.
        sigma (float): Volatility of the stock.
        type (str): Option type, 'c' for call, 'p' for put.

    Returns:
    float: Option price.
    float: Option Vega.
    """
    N = round(5000 * T)
    dt = T / N # time step per layer should tend to 0 to assume continuous time

    a = np.exp(- r * dt) # discount by risk free rate

    # binomial tree pricing parameters
    u = np.exp(sigma * np.sqrt(dt)) # up state
    d = 1 / u # down state is reciprocal of up state
    p = (np.exp(r * dt) - d) / (u - d) # probability of up state

    # vega tree parameters
    # partial derivative of probability of up state to volatility 
    # dp = sqrt(dt) * {(u - d) - [e^(r * dt) - d] * (u + d)} / (u - d) ** 2
    dp = np.sqrt(dt) * (d * (u - d) - (np.exp(r * dt) - d) * (u + d)) / (u - d) ** 2

    # Array of stock prices at maturity, where at time step = N
    # s_ij = S_0 * u^(i) * d^(j) where i + j = N, hence:
    S_t = S0 * u ** np.arange(N,-1,-1) * d ** np.arange(0,N+1,1) # (N+1) x 1 vector

    # array of option vega at maturity, where time step = N
    v_t = np.sqrt(dt) * (2 * np.arange(N,-1,-1) - N) * S0 * np.exp(sigma * np.sqrt(dt) * (2 * np.arange(N,-1,-1) - N))

    # Hence, option prices at time T, the Nth step is as follows, where vega is 0 when f = 0:
    if type == 'c':
        f = np.maximum(S_t - K, 0)
        v = np.where(S_t > K, v_t, 0) 
    elif type == 'p':
        f = np.maximum(K - S_t, 0)
        v = np.where(K > S_t, v_t, 0)
    else:
        raise ValueError("Option type must be 'c' or 'p'.")
    
    # recursive calculation of option price and vega at each time step, from N-1 to 0
    for i in range(N-1, -1, -1):
        S_t = S0 * u ** np.arange(i,-1,-1) * d ** np.arange(0,i+1,1) # N number of outcomes at N-1 step
        
        # value of options at previous step = discount * (p * all up steps * (1-p) * all down steps
        f_previous = a * (p * f[:-1] + (1-p) * f[1:]) # N number of outcomes at N-1 step
        # value of vega at previous step
        v_previous = a * (dp * (f[:-1] - f[1:]) + p * v[:-1] + (1-p) * v[1:])
        # update option price and vega for the loop
        f = f_previous
        v = v_previous

        #  american option has the ability to exercise at any time, it is always worth at least the intrinsic value of the option:
        # S_t - k for call, k - S_t for put
        if type == 'c':
            f = np.maximum(f, S_t - K)
        elif type == 'p':
            f = np.maximum(f, K - S_t)

    return f[0], v[0]


#### IV Derivation Functions

IV can be derived from market data through Brent's root search method, used as the most accurate way. However, it is computationally expensive and slow. Considering that we have many options to calculate IV for, plus time step dt is close to 0, that means that the pricing model will run many times, which will be inefficient but very accurate as Brent's is almost guaranteed to converge.

Hence, using Vega, the first partial derivative of the option price with respect to IV, a faster convergence method in Newton-Ralphson (Newton's) can be used, which may not work sometimes but is much faster than Brent's. A fallback to use Brent's is also included in the case where Newton's does not. For more details, look into Proofs.

In [7]:
def iv_brentq(S0, K, T, r, initial_guess_iv, f, type='c'):
    """
    We solely use Brent's method to find the IV for American options. This is much easier to implement than Newton-Raphson
    but much slower as well.

    The workflow is as such:
    1. Check if the option is worth less than the intrinsic value. If so, return np.nan
    2. Use Brent's method to find the IV
    3. If there is no convergence, return np.nan

    Args:
        S0 (float): The current price of the underlying asset
        K (float): The strike price of the option
        T (float): The time to expiration of the option in years
        r (float): The risk-free interest rate
        initial_guess_iv (float): The initial guess for the IV
        f (float): The current price of the option
        type (str): The type of the option ('c' for call, 'p' for put)

    Returns:
        float: The IV of the option
    """
    if type == 'c':
        intrinsic_value = max(S0 - K, 0)
    elif type == 'p':
        intrinsic_value = max(K - S0, 0)
    if f < intrinsic_value:
        return np.nan # american options cannot be worth less than intrinsic value
    else:
        try:
            iv = brentq(lambda initial_guess_iv: binomial_price(S0, K, T, r, initial_guess_iv, type) - f, 0.001, 2)
            print(f"Brent Converged for S0: {round(S0, 2)}, K: {round(K, 2)}, T: {round(T, 2)}, r: {round(r, 2)}, IV: {round(iv, 2)}, f: {round(f, 2)}")
            return iv
            
        except ValueError:
            print("Brent's method failed, returning np.nan")
            return np.nan

In [8]:
def iv_newton(S0, K, T, r, initial_guess_iv, f, type='c', max_iterations = 200, tolerance = 1e-6):
    """ 
    Newton-Ralphson method can be used to estimate IV for american options due to existence of vega function from
    the backwards induction and automatic differentiation method in the binomial tree.

    The workflow is as such:
    1. Check if the option is valid (f > intrinsic value)
    2. Attempt Newton-Raphson method first.
    3. If Newton-Raphson method fails, use Brent's method as a fallback.
    4. If there is no convergence or Brent's method fails, return np.nan

    Args:
        S0 (float): Current price of the underlying asset
        K (float): Strike price of the option
        T (float): Time to expiration in years
        r (float): Risk-free interest rate
        initial_guess_iv (float): Initial guess for the IV
        f (float): Option price
        type (str, optional): Type of option ('c' for call, 'p' for put). Defaults to 'c'.
        max_iterations (int, optional): Maximum number of iterations for the Newton-Raphson method. Defaults to 200.
        tolerance (float, optional): Tolerance for convergence. Defaults to 1e-6.

    Returns:
        float: Estimated IV for the option
    """

    iv_trial = initial_guess_iv
    # first filter out options with intrinsic value > option value, as american options cannot be worth less than intrinsic value
    if type == 'c':
        intrinsic_value = max(S0 - K, 0)
    elif type == 'p':
        intrinsic_value = max(K - S0, 0)
    if f < intrinsic_value:
        return np.nan
        
    for i in range(max_iterations):
        price_trial, vega_trial = binomial_price_vega_ad(S0, K, T, r, iv_trial, type)
        iv_previous = iv_trial # store value first for convergence check

        try: # fast newton raphson method
            if abs(vega_trial) < 1e-6: # vega can potentially be very small. hence, to prevent division by zero, go into fallback method
                raise ZeroDivisionError("Vega is too small, using Brent's method instead")
            # newton raphson method
            iv_trial = iv_trial - (price_trial - f) / vega_trial # Next IV Estimate = Old IV Estimate - f(iv)/f'(iv)
            # convergence check
            if abs(iv_trial - iv_previous) < tolerance: # convergence check if iv_trial is close enough to the previous iv_trial
                print(f"Newton-Raphson Converged for S0: {round(S0, 2)}, K: {round(K, 2)}, T: {round(T, 2)}, r: {round(r, 2)}, IV: {round(iv_trial, 2)}, f: {round(f, 2)}, vega: {round(vega_trial, 2)}")
                return iv_trial
        # fallback brent method
        except (ValueError, ZeroDivisionError):
            print("ValueError or ZeroDivisionError, Brent's method used instead")
            try:
                iv_trial = brentq(lambda initial_guess_iv: binomial_price_vega_ad(S0, K, T, r, initial_guess_iv, type)[0] - f, 0.001, 2) #only take price
                print(f"Brent Converged for S0: {round(S0, 2)}, K: {round(K, 2)}, T: {round(T, 2)}, r: {round(r, 2)}, IV: {round(iv_trial, 2)}, f: {round(f, 2)}, vega: {round(vega_trial, 2)}")
                return iv_trial
            except ValueError: # if brent method fails, return np.nan
                return np.nan
                print("Brent's method failed, returning np.nan")
    return np.nan # if there is no convergence, return np.nan
    print("No convergence, returning np.nan")

#### Newton's Method

In [9]:
def get_iv_newton(df, rf, type='c'):
    """
    Estimates the implied volatility (IV) for a given option chain using the Newton Ralphson method. This is for use
    with the binomial_pricing_vega_ad and the iv_newton function. Variables are first vectorized for faster
    performance.

    These functions are much faster than only using Brent's method in the iv_brentq function, completing this
    task in less than half the time.
    
    Args:
    df (pd.DataFrame): DataFrame containing option chain data with columns 'S0', 'K', 'T', 'f', and 'rf'.
    rf (float): Risk-free interest rate.
    type (str): Option type ('c' for calls, 'p' for puts).
    
    Returns:
    pd.DataFrame: DataFrame with added 'iv' column containing estimated IV values.
    """
    # make a copy of the df to avoid modifying the original
    df = df.copy()
    
    # vectorize variables
    S0 = df['S0'].values
    K = df['K'].values
    T = df['T'].values
    f = df['f'].values
    df['rf'] = rf

    # vectorize the iv_newton function
    vec_iv_function = np.vectorize(iv_newton)

    df['iv'] = vec_iv_function(S0, K, T, rf, 1, f, type)

    # clean up options where IV is not found
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [10]:
df_iv_newton = get_iv_newton(option_chains, rf)
df_iv_newton

Newton-Raphson Converged for S0: 260.33, K: 225.0, T: 0.06, r: 0.04, IV: 0.84, f: 42.39, vega: 17.05
Newton-Raphson Converged for S0: 260.33, K: 225.0, T: 0.06, r: 0.04, IV: 0.84, f: 42.39, vega: 17.05
Newton-Raphson Converged for S0: 260.33, K: 230.0, T: 0.06, r: 0.04, IV: 0.43, f: 32.16, vega: 10.6
Newton-Raphson Converged for S0: 260.33, K: 235.0, T: 0.06, r: 0.04, IV: 0.32, f: 26.54, vega: 9.08
Newton-Raphson Converged for S0: 260.33, K: 240.0, T: 0.06, r: 0.04, IV: 0.32, f: 22.18, vega: 14.05
Newton-Raphson Converged for S0: 260.33, K: 245.0, T: 0.06, r: 0.04, IV: 0.31, f: 18.0, vega: 17.6
Newton-Raphson Converged for S0: 260.33, K: 250.0, T: 0.06, r: 0.04, IV: 0.3, f: 13.97, vega: 20.84
Newton-Raphson Converged for S0: 260.33, K: 255.0, T: 0.06, r: 0.04, IV: 0.29, f: 10.54, vega: 23.37
Newton-Raphson Converged for S0: 260.33, K: 260.0, T: 0.06, r: 0.04, IV: 0.29, f: 7.6, vega: 24.81
Newton-Raphson Converged for S0: 260.33, K: 265.0, T: 0.06, r: 0.04, IV: 0.28, f: 5.2, vega: 24.23

,K,f,T,Volume,S0,rf,iv
0,225.0,42.39,0.057534,47.0,260.329987,0.0364,0.842709
1,230.0,32.16,0.057534,14.0,260.329987,0.0364,0.429753
2,235.0,26.54,0.057534,120.0,260.329987,0.0364,0.315703
3,240.0,22.18,0.057534,10.0,260.329987,0.0364,0.320208
4,245.0,18.00,0.057534,79.0,260.329987,0.0364,0.314396
...,...,...,...,...,...,...,...
247,320.0,9.20,0.939726,12.0,260.329987,0.0364,0.245097
248,330.0,7.24,0.939726,19.0,260.329987,0.0364,0.242480
249,340.0,5.75,0.939726,27.0,260.329987,0.0364,0.241626
250,350.0,4.56,0.939726,61.0,260.329987,0.0364,0.241227


#### Brent's Method

In [11]:
def get_iv(df, rf, type='c'):
    df = df.copy()
    S0 = df['S0'].values
    K = df['K'].values
    T = df['T'].values
    f = df['f'].values
    df['rf'] = rf

    vec_iv_function = np.vectorize(iv_brentq)

    df['iv'] = vec_iv_function(S0, K, T, rf, 1, f, type)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [12]:
df_iv = get_iv(option_chains, rf)
df_iv

Brent Converged for S0: 260.33, K: 225.0, T: 0.06, r: 0.04, IV: 0.84, f: 42.39
Brent Converged for S0: 260.33, K: 225.0, T: 0.06, r: 0.04, IV: 0.84, f: 42.39
Brent Converged for S0: 260.33, K: 230.0, T: 0.06, r: 0.04, IV: 0.43, f: 32.16
Brent Converged for S0: 260.33, K: 235.0, T: 0.06, r: 0.04, IV: 0.32, f: 26.54
Brent Converged for S0: 260.33, K: 240.0, T: 0.06, r: 0.04, IV: 0.32, f: 22.18
Brent Converged for S0: 260.33, K: 245.0, T: 0.06, r: 0.04, IV: 0.31, f: 18.0
Brent Converged for S0: 260.33, K: 250.0, T: 0.06, r: 0.04, IV: 0.3, f: 13.97
Brent Converged for S0: 260.33, K: 255.0, T: 0.06, r: 0.04, IV: 0.29, f: 10.54
Brent Converged for S0: 260.33, K: 260.0, T: 0.06, r: 0.04, IV: 0.29, f: 7.6
Brent Converged for S0: 260.33, K: 265.0, T: 0.06, r: 0.04, IV: 0.28, f: 5.2
Brent Converged for S0: 260.33, K: 270.0, T: 0.06, r: 0.04, IV: 0.28, f: 3.41
Brent Converged for S0: 260.33, K: 275.0, T: 0.06, r: 0.04, IV: 0.28, f: 2.14
Brent Converged for S0: 260.33, K: 280.0, T: 0.06, r: 0.04, 

,K,f,T,Volume,S0,rf,iv
0,225.0,42.39,0.057534,47.0,260.329987,0.0364,0.842709
1,230.0,32.16,0.057534,14.0,260.329987,0.0364,0.429753
2,235.0,26.54,0.057534,120.0,260.329987,0.0364,0.315703
3,240.0,22.18,0.057534,10.0,260.329987,0.0364,0.320208
4,245.0,18.00,0.057534,79.0,260.329987,0.0364,0.314396
...,...,...,...,...,...,...,...
247,320.0,9.20,0.939726,12.0,260.329987,0.0364,0.245097
248,330.0,7.24,0.939726,19.0,260.329987,0.0364,0.242480
249,340.0,5.75,0.939726,27.0,260.329987,0.0364,0.241626
250,350.0,4.56,0.939726,61.0,260.329987,0.0364,0.241227


### 4. Plot IV Surface

In [13]:
def plot_iv_surface(df):
    df['log_moneyness'] = np.log(df['S0'] / df['K'])

    x = df['log_moneyness'].values
    y = df['T'].values
    z = df['iv'].values

    # Filter out outliers below the 2.5th percentile and above 97.5 percentile.
    mask = (np.abs(z) <= np.percentile(z, 97.5))
    x, y, z = x[mask], y[mask], z[mask]

    # define grid of x = log_moneyness and y = T. high resolution not required.
    x_axis = np.linspace(x.min(), x.max(), 50)
    y_axis = np.linspace(y.min(), y.max(), 50)
    X_interpolated, Y_interpolated = np.meshgrid(x_axis, y_axis)

    # smooth interpolation with cubic method
    Z_interpolated = griddata((x, y), z, (X_interpolated, Y_interpolated), method='linear')

    # plot surface
    fig = go.Figure(data=[go.Surface(
        z = Z_interpolated, 
        x = X_interpolated, 
        y = Y_interpolated,
        colorscale = 'Viridis',
        connectgaps = True
    )])

    fig.update_layout(
        title = 'IV Surface Plot with Interpolation',
        scene = dict(
            xaxis_title = 'Log Moneyness',
            yaxis_title = 'Time to Expiration (T)',
            zaxis_title = 'Implied Volatility'
        ),
        width=900,
        height=800
    )
    
    fig.show()

In [14]:
plot_iv_surface(df_iv)

In [15]:
plot_iv_surface(df_iv_newton)

Looks to be extremely similar. We will use RMSE to check and be certain.

### 5. Evaluation

In [16]:
def rmse_iv(brent_df, newton_df):
    """
    Hence, assuming that Brent's method is the absolute correct method, we evaluate the accuracy of the newton method.
    """
    df1 = brent_df.copy()
    df2 = newton_df.copy()
    # merge to match the options data aside from IV
    rmse_df = df1.merge(df2, on=['S0', 'K', 'T', 'rf'], how='inner')
    rmse_df.dropna(inplace=True) # remove na or missing values in case

    rmse = np.sqrt(mean_squared_error(rmse_df['iv_x'], rmse_df['iv_y']))
    
    print(f"RMSE: {rmse}")

In [17]:
rmse_iv(df_iv, df_iv_newton)

RMSE: 2.1075717792412235e-13


As RMSE is extremely close to 0, it can be concluded that Newton's method would be a better way of computing IV as it is faster and as accurate as Brent's.